In [2]:
dataset_name = 'monuseg'
device = 'cuda'
split_num = 1

import torch
import os
import numpy as np
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.MaskRCNNlib.tools.dataset import MonusegDataset
from Netlib.MaskRCNNlib.Config import mrcnn_configs
from Netlib.MaskRCNNlib.model.GeneralizedRCNN import MRCNN

arg = mrcnn_configs.MrcnnMonuSeg(ds = dataset_name, device = device, ds_num = split_num)
net = arg.model(
            anchors=arg.anchor_wh.to(arg.device),
            backbone=arg.BACKBONE,
            bottom_up=arg.BOTTOM_UP,
            proposal_generator=arg.PROPAOSAL_GENERATOR,
            stride=arg.STRIDE,
            rpn_pos_threshold=arg.RPN_POS_THRESHOLD,
            rpn_fraction_ratio=arg.RPN_FRACTION_RATIO,
            nms_threshold=arg.NMS_THRESHOLD,
            pre_nms_k=arg.PRE_NMS_K,
            post_nms_k=arg.POST_NMS_K,
            roi_head=arg.ROI_HEAD,
            box_detection=arg.BOX_DETECTION,
            expand=arg.EXPAND,
            expand_ratio=arg.EXPAND_RATIO,
            use_gt_box=arg.USE_GT_BOX,
            roi_resolution=arg.ROI_RESOLUTION,
            stage2_max_proposal=arg.STAGE2_MAX_PROPOSAL,
            stage2_sample_ratio=arg.STAGE2_SAMPLE_RATIO,
            box_weight=arg.BOX_WEIGHT,
            roi_pos_threshold=arg.ROI_POS_THRESHOLD,
            post_decttion_score_threshold=arg.POST_DETECTION_SCORE_THRESHOLD,
            detection_per_img=arg.DETECTION_PER_IMG,
            num_classes=arg.NUM_CLASSES,
            use_semantic=arg.USE_SEMANTIC,
            seg_stride=arg.SEG_STRIDE,
            fuse_feature=arg.FUSE_FEATURE,
        ).to(device=arg.device)
net.load_state_dict(torch.load(arg.load_model_para))
net.to(device=device)
net.eval()
data = MonusegDataset(arg.ROI_RESOLUTION,[MyCenterCrop((256,256))],train=False)
preds = np.zeros((0,256,256,2))
for i in range(len(data)):
    imgs = data[i][0].to(device).unsqueeze(0)
    pred, _ = net(imgs)
    pred = net.inference(pred,[256,256])
    preds = np.concatenate([preds,pred],axis=0)


pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score

def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")

#获取文件目录中的父目录路径：
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))

np.save(pred_path,preds)
cp = MyCenterCrop((256,256))

gts = cp((data.labels))[0].permute(0,2,3,1).cpu().numpy()
np.save(gts_path,gts)
imgs = cp((data.imgs))[0]

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)

print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')

Loading data into memory
Finishing loading data
dice score:0.756995
numpy_result/monuseg/1/MrcnnMonuSeg_test_result/gts.npy
numpy_result/monuseg/1/MrcnnMonuSeg_test_result/preds.npy
1


100%|██████████| 48/48 [00:04<00:00, 10.08it/s]


      pq       SQ       DQ      AJI  multi_pq+
0.505399 0.716157 0.703389 0.534207   0.515254


0

In [1]:
dataset_name = 'cpm'
device = 'cuda'
split_num = 1

import torch
import os
import numpy as np
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.MaskRCNNlib.tools.dataset import CPMDataset
from Netlib.MaskRCNNlib.Config import mrcnn_configs
from Netlib.MaskRCNNlib.model.GeneralizedRCNN import MRCNN

arg = mrcnn_configs.MrcnnCPM(ds = dataset_name, device = device, ds_num = split_num)
net = arg.model(
            anchors=arg.anchor_wh.to(arg.device),
            backbone=arg.BACKBONE,
            bottom_up=arg.BOTTOM_UP,
            proposal_generator=arg.PROPAOSAL_GENERATOR,
            stride=arg.STRIDE,
            rpn_pos_threshold=arg.RPN_POS_THRESHOLD,
            rpn_fraction_ratio=arg.RPN_FRACTION_RATIO,
            nms_threshold=arg.NMS_THRESHOLD,
            pre_nms_k=arg.PRE_NMS_K,
            post_nms_k=arg.POST_NMS_K,
            roi_head=arg.ROI_HEAD,
            box_detection=arg.BOX_DETECTION,
            expand=arg.EXPAND,
            expand_ratio=arg.EXPAND_RATIO,
            use_gt_box=arg.USE_GT_BOX,
            roi_resolution=arg.ROI_RESOLUTION,
            stage2_max_proposal=arg.STAGE2_MAX_PROPOSAL,
            stage2_sample_ratio=arg.STAGE2_SAMPLE_RATIO,
            box_weight=arg.BOX_WEIGHT,
            roi_pos_threshold=arg.ROI_POS_THRESHOLD,
            post_decttion_score_threshold=arg.POST_DETECTION_SCORE_THRESHOLD,
            detection_per_img=arg.DETECTION_PER_IMG,
            num_classes=arg.NUM_CLASSES,
            use_semantic=arg.USE_SEMANTIC,
            seg_stride=arg.SEG_STRIDE,
            fuse_feature=arg.FUSE_FEATURE,
        ).to(device=arg.device)
net.load_state_dict(torch.load(arg.load_model_para))
net.to(device=device)
net.eval()
data = CPMDataset(arg.DATASET_FP,arg.ROI_RESOLUTION,[MyCenterCrop((256,256))],train=False)
preds = np.zeros((0,256,256,2))
for i in range(len(data)):
    imgs = data[i][0].to(device).unsqueeze(0)
    pred, _ = net(imgs)
    pred = net.inference(pred,[256,256])
    preds = np.concatenate([preds,pred],axis=0)
    
pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score

def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")

#获取文件目录中的父目录路径：
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))

np.save(pred_path,preds)
cp = MyCenterCrop((256,256))

gts = np.load('data/cpm17/test/labels.npy')
np.save(gts_path,gts)
imgs = torch.load('data/cpm17/test/imgs.pt')

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)

print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')

Loading data into memory
not find the path
dice score:0.826880
numpy_result/cpm/1/MrcnnCPM_test_result/gts.npy
numpy_result/cpm/1/MrcnnCPM_test_result/preds.npy
1


100%|██████████| 32/32 [00:02<00:00, 15.89it/s]


      pq       SQ      DQ      AJI  multi_pq+
0.609784 0.763688 0.79655 0.642788   0.610986


0

In [3]:
dataset_name = 'dsb'
device = 'cuda'
split_num = 1

import torch
import os
import numpy as np
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.MaskRCNNlib.tools.dataset import DSBDataset
from Netlib.MaskRCNNlib.Config import mrcnn_configs
from Netlib.MaskRCNNlib.model.GeneralizedRCNN import MRCNN

arg = mrcnn_configs.MrcnnDSB(ds=dataset_name,device=device,ds_num=split_num)
net = arg.model(
            anchors=arg.anchor_wh.to(arg.device),
            backbone=arg.BACKBONE,
            bottom_up=arg.BOTTOM_UP,
            proposal_generator=arg.PROPAOSAL_GENERATOR,
            stride=arg.STRIDE,
            rpn_pos_threshold=arg.RPN_POS_THRESHOLD,
            rpn_fraction_ratio=arg.RPN_FRACTION_RATIO,
            nms_threshold=arg.NMS_THRESHOLD,
            pre_nms_k=arg.PRE_NMS_K,
            post_nms_k=arg.POST_NMS_K,
            roi_head=arg.ROI_HEAD,
            box_detection=arg.BOX_DETECTION,
            expand=arg.EXPAND,
            expand_ratio=arg.EXPAND_RATIO,
            use_gt_box=arg.USE_GT_BOX,
            roi_resolution=arg.ROI_RESOLUTION,
            stage2_max_proposal=arg.STAGE2_MAX_PROPOSAL,
            stage2_sample_ratio=arg.STAGE2_SAMPLE_RATIO,
            box_weight=arg.BOX_WEIGHT,
            roi_pos_threshold=arg.ROI_POS_THRESHOLD,
            post_decttion_score_threshold=arg.POST_DETECTION_SCORE_THRESHOLD,
            detection_per_img=arg.DETECTION_PER_IMG,
            num_classes=arg.NUM_CLASSES,
            use_semantic=arg.USE_SEMANTIC,
            seg_stride=arg.SEG_STRIDE,
            fuse_feature=arg.FUSE_FEATURE,
        ).to(device=arg.device)
net.load_state_dict(torch.load(arg.load_model_para))
net.to(device=device)
net.eval()
data = DSBDataset(arg.DATASET_FP,transfs=[MyCenterCrop((256,256))],train=True)
preds = np.zeros((0,256,256,2),dtype=np.int32)

for i in arg.TEST_SET_INDEX:
    img = data[i][0].unsqueeze(0).to(device)
    pred, _ = net(img)
    pred = net.inference(pred,[256,256]) 
    preds = np.concatenate([preds,pred],axis=0)

pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))

import PIL.Image as Image
from torchvision.transforms.functional import pil_to_tensor
from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score

def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")
cp = MyCenterCrop((256,256))
np.save(pred_path,preds)
print(arg.TEST_SET_INDEX)
imgs = torch.zeros((0,3,256,256),dtype=torch.float32)
gts = np.zeros((0,256,256,2),dtype=np.int32)

for i in arg.TEST_SET_INDEX:
    img = Image.open(os.path.join('data/DSB2018/train', data.data_dir[i], 'images', data.data_dir[i]+'.png')).convert('RGB')
    img = pil_to_tensor(img).float() / 255
    img = cp((img))[0]
    instances = os.listdir(os.path.join( 'data/DSB2018/train',data.data_dir[i], "masks"))
    instances.sort()
    label = torch.zeros(2, img.shape[1], img.shape[2], dtype=torch.long)
    for idx, ins in enumerate(instances):
        mask = Image.open(os.path.join(data.train_dir, data.data_dir[i], "masks", ins))
        mask = pil_to_tensor(mask).bool() * 1
        mask = cp((mask))[0]
        label[0][label[0]==0] += mask[0][label[0]==0] * (idx+1)
        label[1][label[1]==0] += mask[0][label[1]==0].long()

    imgs = torch.cat((imgs,img.unsqueeze(0)),0)
    gts = np.concatenate((gts,label.permute(1,2,0).unsqueeze(0).cpu().numpy()),axis=0)

np.save(gts_path,gts)

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)

print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')


Loading data into memory
tensor([659, 439, 334, 558, 271, 414, 195, 652,  23, 409, 292, 111, 287, 324,
        592, 390, 612, 648, 340, 615, 660, 108,   2, 223, 134, 495,  46,  15,
        275, 498, 216, 132, 500, 163, 123, 452,  51, 341, 376, 370, 655,  24,
        211,  80, 526, 496, 446, 385, 173, 548, 471, 561, 209, 305,  37, 551,
        247, 588, 176, 604, 233, 189,  68, 333, 515,  42, 208, 139, 194,  69,
        432, 627, 589, 295, 595, 465, 646,  34,  38, 383, 162,  57, 538,  64,
        477, 613,  85, 605,   8, 213, 510, 293, 265, 261, 210, 481, 356, 408,
        323, 583,  75, 528, 336, 621,  43, 235, 529, 568, 180, 114,  36, 364,
        405, 620, 144, 351, 168, 472,  20, 120, 518, 177, 397, 142, 191, 278,
        649, 299, 447, 463, 601, 519, 527,  56])
not find the path
dice score:0.865691
numpy_result/dsb/1/MrcnnDSB_test_result/gts.npy
numpy_result/dsb/1/MrcnnDSB_test_result/preds.npy
1


100%|██████████| 134/134 [00:05<00:00, 25.85it/s]


      pq       SQ       DQ      AJI  multi_pq+
0.703093 0.820552 0.845341 0.710505   0.681118


0